### Moive Recommendation
In this notebook, we will use an Alternating Least Squares (ALS) algorithm with Spark APIs to predict the ratings for the movies.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [3]:
import os
os.environ["PYSPARK_PYTHON"] = "python3"

## Part1: Data ETL and Data Exploration

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("moive analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [6]:
movies_df = spark.read.load("/FileStore/tables/movies.csv", format='csv', header = True)
ratings_df = spark.read.load("/FileStore/tables/ratings.csv", format='csv', header = True)
links_df = spark.read.load("/FileStore/tables/links.csv", format='csv', header = True)
tags_df = spark.read.load("/FileStore/tables/tags.csv", format='csv', header = True)

In [7]:
movies_df.show(5)

+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1| Toy Story (1995)|Adventure|Animati...|
 2| Jumanji (1995)|Adventure|Childre...|
 3|Grumpier Old Men ...| Comedy|Romance|
 4|Waiting to Exhale...|Comedy|Drama|Romance|
 5|Father of the Bri...| Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

In [8]:
ratings_df.show(100)

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
 1| 70| 3.0|964982400|
 1| 101| 5.0|964980868|
 1| 110| 4.0|964982176|
 1| 151| 5.0|964984041|
 1| 157| 5.0|964984100|
 1| 163| 5.0|964983650|
 1| 216| 5.0|964981208|
 1| 223| 3.0|964980985|
 1| 231| 5.0|964981179|
 1| 235| 4.0|964980908|
 1| 260| 5.0|964981680|
 1| 296| 3.0|964982967|
 1| 316| 3.0|964982310|
 1| 333| 5.0|964981179|
 1| 349| 4.0|964982563|
 1| 356| 4.0|964980962|
 1| 362| 5.0|964982588|
 1| 367| 4.0|964981710|
 1| 423| 3.0|964982363|
 1| 441| 4.0|964980868|
 1| 457| 5.0|964981909|
 1| 480| 4.0|964982346|
 1| 500| 3.0|964981208|
 1| 527| 5.0|964984002|
 1| 543| 4.0|964981179|
 1| 552| 4.0|964982653|
 1| 553| 5.0|964984153|
 1| 590| 4.0|964982546|
 1| 592| 4.0|964982271|
 1| 593| 4.0|964983793|
 1| 596| 5.0|964982838|
 1| 608| 5.0|964982931|
 1| 648| 3.0|964982563|
 1| 661| 5.0|964982838|
 1| 673| 3.0|964981775|
 1| 733| 4.0|964982400|
 1| 736| 3.0|964982653|
 1| 780| 3.0|964984086|
 1| 804| 4.0|964980499|
 1| 919| 5.0|964982475|
 1| 923| 5.0|964981529|
 1| 940| 5.0|964982176|
 1| 943| 4.0|964983614|
 1| 954| 5.0|964983219|
 1| 1009| 3.0|964981775|
 1| 1023| 5.0|964982681|
 1| 1024| 5.0|964982876|
 1| 1025| 5.0|964982791|
 1| 1029| 5.0|964982855|
 1| 1030| 3.0|964982903|
 1| 1031| 5.0|964982653|
 1| 1032| 5.0|964982791|
 1| 1042| 4.0|964981179|
 1| 1049| 5.0|964982400|
 1| 1060| 4.0|964980924|
 1| 1073| 5.0|964981680|
 1| 1080| 5.0|964981327|
 1| 1089| 5.0|964982951|
 1| 1090| 4.0|964984018|
 1| 1092| 5.0|964983484|
 1| 1097| 5.0|964981680|
 1| 1127| 4.0|964982513|
 1| 1136| 5.0|964981327|
 1| 1196| 5.0|964981827|
 1| 1197| 5.0|964981872|
 1| 1198| 5.0|964981827|
 1| 1206| 5.0|964983737|
 1| 1208| 4.0|964983250|
 1| 1210| 5.0|964980499|
 1| 1213| 5.0|964982951|
 1| 1214| 4.0|964981855|
 1| 1219| 2.0|964983393|
 1| 1220| 5.0|964981909|
 1| 1222| 5.0|964981909|
 1| 1224| 5.0|964984018|
 1| 1226| 5.0|964983618|
 1| 1240| 5.0|964983723|
 1| 1256| 5.0|964981442|
 1| 1258| 3.0|964983414|
 1| 1265| 4.0|964983599|
 1| 1270| 5.0|964983705|
 1| 1275| 5.0|964982290|
 1| 1278| 5.0|964983414|
 1| 1282| 5.0|964982703|
 1| 1291| 5.0|964981909|
 1| 1298| 5.0|964984086|
 1| 1348| 4.0|964983393|
 1| 1377| 3.0|964982653|
 1| 1396| 3.0|964983017|
 1| 1408| 3.0|964982310|
 1| 1445| 3.0|964984112|
 1| 1473| 4.0|964980875|
 1| 1500| 4.0|964980985|
 1| 1517| 5.0|964981107|
 1| 1552| 4.0|964982620|
+------+-------+------+---------+
only showing top 100 rows

In [9]:
links_df.show(5)

+-------+-------+------+
movieId| imdbId|tmdbId|
+-------+-------+------+
 1|0114709| 862|
 2|0113497| 8844|
 3|0113228| 15602|
 4|0114885| 31357|
 5|0113041| 11862|
+-------+-------+------+
only showing top 5 rows

In [10]:
tags_df.show(5)

+------+-------+---------------+----------+
userId|movieId| tag| timestamp|
+------+-------+---------------+----------+
 2| 60756| funny|1445714994|
 2| 60756|Highly quotable|1445714996|
 2| 60756| will ferrell|1445714992|
 2| 89774| Boxing story|1445715207|
 2| 89774| MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows

In [11]:
tmp1 = ratings_df.groupBy("userID").count().toPandas()['count'].min()
tmp2 = ratings_df.groupBy("movieId").count().toPandas()['count'].min()
print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1))
print('Minimum number of ratings per movie is {}'.format(tmp2))

For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 20
Minimum number of ratings per movie is 1

In [12]:
tmp1 = sum(ratings_df.groupBy("movieId").count().toPandas()['count'] == 1)
tmp2 = ratings_df.select('movieId').distinct().count()
print('{} out of {} movies are rated by only one user'.format(tmp1, tmp2))

3446 out of 9724 movies are rated by only one user

## Part 1: Spark SQL and OLAP

In [14]:
movies_df.registerTempTable("movies")
ratings_df.registerTempTable("ratings")
links_df.registerTempTable("links")
tags_df.registerTempTable("tags")


### Q1: The number of Users

In [16]:
%sql 
Select Count(Distinct userId) as Number_of_users From ratings

Number_of_users
610


### Q2: The number of Movies

In [18]:
%sql 
Select Count(Distinct movieId) as Number_of_movies From movies

Number_of_movies
9742


### Q3:  How many movies are rated by users? List movies not rated before

In [20]:
%sql
 
Select title, genres from movies  
where movieId not in 
  (Select movieId from ratings)



title,genres
"Innocents, The (1961)",Drama|Horror|Thriller
Niagara (1953),Drama|Thriller
For All Mankind (1989),Documentary
"Color of Paradise, The (Rang-e khoda) (1999)",Drama
I Know Where I'm Going! (1945),Drama|Romance|War
"Chosen, The (1981)",Drama
"Road Home, The (Wo de fu qin mu qin) (1999)",Drama|Romance
Scrooge (1970),Drama|Fantasy|Musical
Proof (1991),Comedy|Drama|Romance
"Parallax View, The (1974)",Thriller


In [21]:
%sql 
Select Count(Distinct movieID) as Number_movies_are_rated_by_users From ratings

Number_movies_are_rated_by_users
9724


### Q4: List Movie Genres

In [23]:
%sql
Select Distinct genres From movies

genres
Comedy|Horror|Thriller
Adventure|Sci-Fi|Thriller
Action|Adventure|Drama|Fantasy
Action|Drama|Horror
Action|Animation|Comedy|Sci-Fi
Animation|Children|Drama|Musical|Romance
Action|Adventure|Drama
Adventure|Sci-Fi
Documentary|Musical|IMAX
Adventure|Children|Fantasy|Sci-Fi|Thriller


In [24]:
%sql
SELECT distinct explode(split(genres,'[|]')) as category
FROM movies 


category
Crime
Romance
Thriller
Adventure
Drama
War
Documentary
Fantasy
Mystery
Musical


### Q5: Movie for Each Category

In [26]:
%sql
Select category, count(distinct movieID) as number
From
(
SELECT movieID, title, explode(split(genres,'[|]')) as category
FROM movies
) as cte
Group by category
order by count(distinct movieID)



category,number
(no genres listed),34
Film-Noir,87
IMAX,158
Western,167
Musical,334
War,382
Documentary,440
Mystery,573
Animation,611
Children,664


In [27]:
%sql 

Select category, collect_list(distinct title) as movies
From
(
SELECT movieID, title, explode(split(genres,'[|]')) as category
FROM movies
) as cte
Group by category
order by category

category movies (no genres listed) List(Guardians (2016), Paterson, Hyena Road, A Cosmic Christmas (1977), Too Funny to Fail: The Life and Death of The Dana Carvey Show (2017), A Christmas Story Live! (2017), The Adventures of Sherlock Holmes and Doctor Watson, A Midsummer Night's Dream (2016), Death Note: Desu nôto (2006–2007), Pirates of the Caribbean: Dead Men Tell No Tales (2017), Whiplash (2013), The Putin Interviews (2017), T2 3-D: Battle Across Time (1996), Green Room (2015), Grease Live (2016), La cravate (1957), Maria Bamford: Old Baby, The Godfather Trilogy: 1972-1990 (1992), Cosmos, Let It Be Me (1995), Lemonade (2016), Ethel & Ernest (2016), The Brand New Testament (2015), Noin 7 veljestä (1968), The Adventures of Sherlock Holmes and Doctor Watson: The Hunt for the Tiger (1980), Superfast! (2015), Ali Wong: Baby Cobra (2016), The Forbidden Dance (1990), Serving in Silence: The Margarethe Cammermeyer Story (1995), Trevor Noah: African American (2013), Ben-hur (2016), Generation Iron 2, The OA, Black Mirror) Action List(Superman III (1983), Impostor (2002), Die Another Day (2002), Italian Job, The (2003), RocknRolla (2008), Daybreakers (2010), Book of Eli, The (2010), Snow White and the Huntsman (2012), Captain America: Civil War (2016), Iron Man & Hulk: Heroes United (2013), Ryuzo and the Seven Henchmen (2015), Ride Along 2 (2016), Star Wars: Episode III - Revenge of the Sith (2005), Day Watch (Dnevnoy dozor) (2006), Hell Ride (2008), A-Team, The (2010), Hot Pursuit (2015), Bright (2017), Men in Black (a.k.a. MIB) (1997), Dragonslayer (1981), Clash of the Titans (1981), Taxi (2004), Wraith, The (1986), Mortadelo & Filemon: The Big Adventure (La gran aventura de Mortadelo y Filemón) (2003), Homefront (2013), London Has Fallen (2016), Dredd (2012), LEGO Batman: The Movie - DC Heroes Unite (2013), Gen-X Cops (1999), Treasure of the Sierra Madre, The (1948), Assignment, The (1997), eXistenZ (1999), Sanjuro (Tsubaki Sanjûrô) (1962), Patriot Games (1992), Another 48 Hrs. (1990), Ice Harvest, The (2005), 300 (2007), MacGruber (2010), Ghost in the Shell: Stand Alone Complex - The Laughing Man (2005), Bad Asses on the Bayou (2015), Judge Dredd (1995), Mad Max (1979), Casino Royale (1967), Bandidas (2006), Dead Fury (2008), Last Airbender, The (2010), Evangelion: 3.0 You Can (Not) Redo (2012), Band of Brothers (2001), Natural Born Killers (1994), Battle Creek Brawl (Big Brawl, The) (1980), Four Musketeers, The (1974), Crank (2006), Ghost Rider (2007), Tekkonkinkreet (Tekkon kinkurîto) (2006), Terminator Salvation (2009), Gravity (2013), Maze Runner, The (2014), The Last Witch Hunter (2015), Freddy vs. Jason (2003), Where Eagles Dare (1968), Paycheck (2003), Fay Grim (2006), Watchmen (2009), Justice League: Doom (2012) , Escape Plan (2013), The Flash 2 - Revenge of the Trickster (1991), Maximum Ride (2016), Love and a .45 (1994), King Kong vs. Godzilla (Kingukongu tai Gojira) (1962), Moonraker (1979), Big Trouble in Little China (1986), Extreme Days (2001), Godzilla vs. Mothra (Mosura tai Gojira) (1964), Public Enemy, The (1931), Superman/Batman: Public Enemies (2009), Fullmetal Alchemist: The Sacred Star of Milos (2011), Midnight Chronicles (2009), Stretch (2014), Mortal Kombat: The Journey Begins (1995), Punisher, The (1989), Kangaroo Jack (2003), Good, the Bad and the Ugly, The (Buono, il brutto, il cattivo, Il) (1966), Boot, Das (Boat, The) (1981), Blade (1998), For a Few Dollars More (Per qualche dollaro in più) (1965), Kiss of the Dragon (2001), Terminator 3: Rise of the Machines (2003), Touch of Zen, A (Xia nu) (1971), Rango (2011), Lockout (2012), Good Day to Die Hard, A (2013), Need for Speed (2014), In the Name of the King III (2014), Army of Darkness (1993), Speed 2: Cruise Control (1997), Armour of God II: Operation Condor (Operation Condor) (Fei ying gai wak) (1991), Innerspace (1987), Troy (2004), Team America: World Police (2004), Hitman (2007), Echelon Conspiracy (2009), Repo Men (2010), The Hitman's Bodyguard 

## Part2: Spark ALS based approach for training model
We will use an Spark ML to predict the ratings, so let's reload "ratings.csv" using ``sc.textFile`` and then convert it to the form of (user, item, rating) tuples.

In [29]:
ratings_df.show()

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
 1| 70| 3.0|964982400|
 1| 101| 5.0|964980868|
 1| 110| 4.0|964982176|
 1| 151| 5.0|964984041|
 1| 157| 5.0|964984100|
 1| 163| 5.0|964983650|
 1| 216| 5.0|964981208|
 1| 223| 3.0|964980985|
 1| 231| 5.0|964981179|
 1| 235| 4.0|964980908|
 1| 260| 5.0|964981680|
 1| 296| 3.0|964982967|
 1| 316| 3.0|964982310|
 1| 333| 5.0|964981179|
 1| 349| 4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows

In [30]:
movie_ratings=ratings_df.drop('timestamp')

In [31]:
# Data type convert
from pyspark.sql.types import IntegerType, FloatType
movie_ratings = movie_ratings.withColumn("userId", movie_ratings["userId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("movieId", movie_ratings["movieId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("rating", movie_ratings["rating"].cast(FloatType()))

In [32]:
movie_ratings.show()

+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 1| 1| 4.0|
 1| 3| 4.0|
 1| 6| 4.0|
 1| 47| 5.0|
 1| 50| 5.0|
 1| 70| 3.0|
 1| 101| 5.0|
 1| 110| 4.0|
 1| 151| 5.0|
 1| 157| 5.0|
 1| 163| 5.0|
 1| 216| 5.0|
 1| 223| 3.0|
 1| 231| 5.0|
 1| 235| 4.0|
 1| 260| 5.0|
 1| 296| 3.0|
 1| 316| 3.0|
 1| 333| 5.0|
 1| 349| 4.0|
+------+-------+------+
only showing top 20 rows

### ALS Model Selection and Evaluation

With the ALS model, we can use a grid search to find the optimal hyperparameters.

In [34]:
# import package
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder

In [35]:
#Create test and train set
(training,test)=movie_ratings.randomSplit([0.8,0.2])

In [36]:
#Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol ="rating", coldStartStrategy = "drop", nonnegative = True)

In [37]:
#Tune model using ParamGridBuilder
param_grid = ParamGridBuilder().addGrid(als.rank, [10,11,12]).addGrid(als.maxIter, [5,7,10]).addGrid(als.regParam, [0.1, 0.01, 0.001]).build()

In [38]:
param_grid = ParamGridBuilder().addGrid(als.rank, [10,11,12]).addGrid(als.maxIter, [5,7,10]).addGrid(als.regParam, [0.1, 0.01, 0.001]).build()
evaluator = RegressionEvaluator(metricName= "rmse", labelCol="rating", predictionCol="prediction")
crossval = CrossValidator(estimator=als,
                          estimatorParamMaps=param_grid,
                          evaluator=evaluator,
                          numFolds=5)

In [39]:
# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName= "rmse", labelCol="rating", predictionCol="prediction")


In [40]:
# Build Cross validation 
crossval = CrossValidator(estimator=als,
                          estimatorParamMaps=param_grid,
                          evaluator=evaluator,
                          numFolds=5)


In [41]:
#Fit ALS model to training data
model = crossval.fit(training)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [42]:
model = crossval.fit(training)
best_model = model.bestModel

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [43]:
#Extract best model from the tuning exercise using ParamGridBuilder
best_model = model.bestModel

### Model testing
And finally, make a prediction and check the testing error.

In [45]:
#Generate predictions and evaluate using RMSE
predictions=best_model.transform(test)
rmse = evaluator.evaluate(predictions)

In [46]:
#Print evaluation metrics and model parameters
print ("RMSE = "+str(rmse))
print ("**Best Model**")
print (" Rank:" +str(best_model.rank))
print (" MaxIter:" +str(best_model._java_obj.parent().getMaxIter()))
print (" RegParam:" + str(best_model._java_obj.parent().getRegParam()))

RMSE = 0.8696467860575481
**Best Model**
 Rank:12
 MaxIter:10
 RegParam:0.1

In [47]:
predictions.show()

+------+-------+------+----------+
userId|movieId|rating|prediction|
+------+-------+------+----------+
 599| 471| 2.5| 2.6550376|
 409| 471| 3.0| 3.7558448|
 387| 471| 3.0| 3.0120077|
 448| 471| 4.0| 3.1823394|
 541| 471| 3.0| 3.989418|
 104| 471| 4.5| 3.8197446|
 609| 833| 3.0| 1.1993757|
 492| 833| 4.0| 1.3983767|
 599| 1088| 2.5| 2.3889964|
 414| 1088| 3.0| 2.9139524|
 517| 1088| 1.0| 2.876779|
 177| 1088| 3.5| 3.8400588|
 84| 1088| 3.0| 3.3194344|
 554| 1088| 5.0| 4.1464996|
 594| 1088| 4.5| 4.5328407|
 525| 1088| 4.5| 3.3773742|
 587| 1238| 4.0| 3.094621|
 216| 1238| 5.0| 3.3964276|
 387| 1342| 3.0| 2.5852082|
 19| 1342| 2.0| 2.721101|
+------+-------+------+----------+
only showing top 20 rows

### Model apply and see the performance

In [49]:
alldata=best_model.transform(movie_ratings)
rmse = evaluator.evaluate(alldata)
print ("RMSE = "+str(rmse))

RMSE = 0.6376846849381629

In [50]:
alldata.registerTempTable("alldata")

In [51]:
%sql select * from alldata

userId,movieId,rating,prediction
191,148,5.0,4.896819
372,471,3.0,2.9016063
603,471,4.0,3.3372178
136,471,4.0,3.9194884
469,471,5.0,3.923933
414,471,5.0,3.4931886
608,471,1.5,2.4006622
104,471,4.5,3.8197446
133,471,4.0,3.3036246
597,471,2.0,3.6391363


In [52]:
%sql select * from movies join alldata on movies.movieId=alldata.movieId

movieId,title,genres,userId,movieId,rating,prediction
148,"Awfully Big Adventure, An (1995)",Drama,191,148,5.0,4.896819
471,"Hudsucker Proxy, The (1994)",Comedy,91,471,1.0,2.4851396
471,"Hudsucker Proxy, The (1994)",Comedy,409,471,3.0,3.7558448
471,"Hudsucker Proxy, The (1994)",Comedy,182,471,4.5,4.310622
471,"Hudsucker Proxy, The (1994)",Comedy,462,471,2.5,2.4522321
471,"Hudsucker Proxy, The (1994)",Comedy,217,471,2.0,2.9235692
471,"Hudsucker Proxy, The (1994)",Comedy,216,471,3.0,2.8851533
471,"Hudsucker Proxy, The (1994)",Comedy,469,471,5.0,3.923933
471,"Hudsucker Proxy, The (1994)",Comedy,373,471,5.0,4.080469
471,"Hudsucker Proxy, The (1994)",Comedy,357,471,3.5,3.9817202


## Recommend moive to users with id: 575, 232. 
you can choose some users to recommend the moives

In [54]:
userRecs = best_model.recommendForAllUsers(10)
display(userRecs)


userId,recommendations
463,"List(List(3200, 4.911063), List(3022, 4.7410994), List(171495, 4.691776), List(184245, 4.691776), List(26073, 4.691776), List(72171, 4.691776), List(84273, 4.691776), List(117531, 4.691776), List(7071, 4.691776), List(179135, 4.691776))"
148,"List(List(87234, 4.7610297), List(98491, 4.4837065), List(160718, 4.3590226), List(3521, 4.3576064), List(84847, 4.315737), List(96821, 4.309941), List(177593, 4.301182), List(93008, 4.2998586), List(77846, 4.2998586), List(25906, 4.2998586))"
471,"List(List(177593, 4.865229), List(8477, 4.6759124), List(3200, 4.6169186), List(96004, 4.480244), List(2324, 4.447288), List(158966, 4.4352326), List(92475, 4.4244566), List(7361, 4.4241776), List(3567, 4.4173255), List(104879, 4.393123))"
496,"List(List(89904, 5.164974), List(213, 4.6993084), List(32582, 4.6654897), List(60766, 4.6065784), List(55276, 4.6001487), List(4429, 4.4809093), List(6639, 4.4630513), List(6201, 4.4542937), List(4495, 4.4542937), List(5747, 4.4468584))"
540,"List(List(100714, 5.103637), List(69524, 5.0196033), List(171495, 4.999335), List(179135, 4.999335), List(7071, 4.999335), List(72171, 4.999335), List(26073, 4.999335), List(84273, 4.999335), List(117531, 4.999335), List(184245, 4.999335))"
243,"List(List(86377, 5.4937325), List(87234, 5.353112), List(86347, 5.330023), List(945, 5.2941575), List(5075, 5.2465158), List(8477, 5.1534834), List(3200, 5.1508293), List(7008, 5.1299267), List(26073, 5.122835), List(7071, 5.122835))"
392,"List(List(5490, 5.0208697), List(5915, 4.9136357), List(3494, 4.8786855), List(1250, 4.873524), List(858, 4.8532043), List(2300, 4.8392177), List(260, 4.802099), List(174055, 4.7915807), List(3508, 4.7770834), List(112552, 4.7716937))"
31,"List(List(51931, 5.6546693), List(2843, 5.6156816), List(177593, 5.399124), List(1046, 5.3527255), List(179135, 5.3485436), List(84273, 5.3485436), List(26073, 5.3485436), List(117531, 5.3485436), List(7071, 5.3485436), List(72171, 5.3485436))"
516,"List(List(89904, 4.8544097), List(4429, 4.809765), List(1046, 4.5961514), List(6201, 4.5700064), List(4495, 4.5700064), List(27523, 4.5651402), List(25825, 4.5564113), List(898, 4.51829), List(177593, 4.5164113), List(5915, 4.516195))"
251,"List(List(96004, 5.406329), List(3022, 5.3960986), List(26171, 5.3907647), List(177593, 5.3784337), List(2131, 5.327105), List(26471, 5.2744155), List(92475, 5.2646847), List(27156, 5.249886), List(5915, 5.246266), List(1178, 5.2420692))"


In [55]:
userRecs.createOrReplaceTempView("userRec")
temp = spark.sql("SELECT userId, recommendations FROM userRec where userId = 575")
display(temp)

userId,recommendations
575,"List(List(8477, 5.420325), List(183897, 5.228679), List(86781, 5.1137958), List(177593, 5.080568), List(148881, 5.067505), List(2303, 5.021811), List(7025, 5.002827), List(188751, 4.9677463), List(176371, 4.9407716), List(58301, 4.9085007))"


In [56]:
userRecs.createOrReplaceTempView("userRec")
temp = spark.sql("SELECT userId, recommendations FROM userRec where userId = 232")

display(temp)

userId,recommendations
232,"List(List(3200, 4.697607), List(5075, 4.654945), List(171495, 4.548487), List(84273, 4.548487), List(26073, 4.548487), List(72171, 4.548487), List(117531, 4.548487), List(179135, 4.548487), List(7071, 4.548487), List(184245, 4.548487))"


## Find the similar moives for moive with id: 463, 471
You can find the similar moives based on the ALS results

In [58]:
factors=best_model.itemFactors
display(factors)

id,features
10,"List(0.29814208, 0.36530912, 0.1235385, 1.3191936, 0.41615722, 0.049333323, 0.7169036, 0.66238296, 0.58697015, 0.4086679, 0.30274802, 0.24906948)"
20,"List(0.5149267, 0.4681858, 0.66447353, 0.024519358, 0.2475168, 0.0, 0.49447954, 0.7850115, 0.3085634, 0.38451654, 1.2286403, 0.0)"
30,"List(1.9744086, 0.0, 0.6769193, 0.0, 0.0, 0.1134667, 0.44481862, 0.5916831, 0.23605524, 0.0, 0.0, 1.018136)"
40,"List(0.14861526, 0.018709095, 0.8189067, 1.1998013, 0.34267634, 0.54863214, 0.00358345, 1.2906394, 0.009430693, 0.5723096, 0.08932574, 0.5129575)"
50,"List(0.7055091, 0.38433766, 1.0771112, 0.97160816, 0.47269776, 0.18892238, 0.4336216, 0.9773563, 0.8559772, 0.4103268, 0.29588607, 0.3446865)"
60,"List(0.37999052, 0.49173614, 0.18709163, 0.642562, 0.42471606, 0.49961355, 0.39475325, 0.832857, 0.23314984, 0.757091, 0.8381465, 0.29273462)"
70,"List(0.26945016, 0.40862557, 0.08552736, 0.90924174, 0.32633874, 0.34024343, 0.56268543, 0.89316005, 0.98951846, 0.19658491, 1.025683, 0.0)"
80,"List(0.56168956, 0.16450606, 0.9886159, 0.5288544, 0.33633593, 0.47990605, 0.642177, 0.5978784, 0.67271215, 0.4380014, 0.3626409, 0.6309376)"
100,"List(0.22152518, 0.7228624, 0.37474236, 0.38841593, 0.35063666, 0.5904562, 0.51418054, 0.2201264, 0.1676978, 0.5865636, 0.22820547, 0.5933339)"
110,"List(0.2862576, 0.3276106, 0.9418058, 1.0464691, 0.26508057, 0.0, 0.7774756, 0.70251465, 0.54621005, 0.975931, 0.35623717, 0.27825004)"


In [59]:
factors.filter(col('id') == 463).show()

+---+--------+
 id|features|
+---+--------+
+---+--------+

In [60]:
factors.filter(col('id') == 471).show()

+---+--------------------+
 id| features|
+---+--------------------+
471|[0.3588241, 0.456...|
+---+--------------------+

In [61]:

import numpy as np


In [62]:
def similar_movies(factors, movieId):

  try: 
    target_id_feature = np.array(factors.filter(col('id') == movieId).select("features").collect())[0]
  except:
    return 'There is no movie with id ' + str(movieId)

  similarities = []
  for feature in np.array(factors.select("features").collect()):
    feature = feature.reshape(12)
    target_id_feature = target_id_feature.reshape(12)
    similarity = np.dot(target_id_feature,feature)/(np.linalg.norm(target_id_feature) * np.linalg.norm(feature))
    similarities.append(similarity)
  
  index = np.array(factors.select("id").collect())
  np_s = np.array(similarities).reshape(8948,1)
  combined_data = np.zeros([8948,2])
  combined_data.shape
  combined_data[:,0]=index[:,0]
  combined_data[:,1]=np_s[:,0]
  pdf = pd.DataFrame(combined_data)
  pdf.columns=['movie index','similarity']
  
  # top 11 similar movies contain the movie itself with similarity = 1, so I need to remove it. 
  top10_movie = pdf.nlargest(11, 'similarity')
  
  return top10_movie

In [63]:
similar_movies(factors, 471)


,movie index,similarity
934,471.0,1.000000
994,1271.0,0.939895
8133,639.0,0.937794
3301,44633.0,0.936442
8693,58309.0,0.936442
5932,26366.0,0.936442
7502,4308.0,0.935694
946,661.0,0.930107
6388,1307.0,0.927974
6960,72737.0,0.925677


In [64]:
similar_movies(factors, 463)

Out[155]: 'There is no movie with id 463'

****Project Outlines***

When we are doing online shopping or visiting video websites, like Amazon and Hulu, we always see a list named “You may also like”. I am very curious about how the online shopping or video platform come up with this list. And I want to use the data analysis technique to recommend products to different users and find out the similar products for the specific products.
1. At first, I download the movie data and create four data frames, movies_df,  ratings_df, links_df and tags-df. And I use some basic methods to give a brief glance of the data.
2. After that, I used spark SQL to do OLAP of the data. I found out the total number of users is 610 and the total number of movies is 9742. After some SQL operation, we can see there are around 20 movies which have no rating. And I also used spark SQL to build a table with lists of all movies for each category.
3. Then we used park ALS to train the movie data and hopefully we can use this model to predict all the ratings for all users and find out the similar movies.  First, we import the evaluator, ALS and the tuning packages. Then we create test and train set. After that, we turn the model using paramGridBuilder. Then we fit the model by cross validation. Then we extract best model, generate predictions and evaluate using RMSE. After model testing, we apply the model to see the performance. 
4. Finally, we can use the model to recommend the movies to specific users. Most importantly, using the dot production from the features matrix, which we extracted from the ALS models, we can find out the similarities among diferent movie items. This feature is very useful when customers want to explode the similar products based on their existing favorite products.

This ALS model can be useful to recommend movie to a current user. However, it has a coldstart issue and it is not applicable for a new user with no former information.